## Summary

```bash
export NOTEBOOK_PATH="$(realpath 38_cagi6_sherloc_train_model.ipynb)"

export FEATURES_TO_USE="proteinsolver-protbert-rosetta_ddg-alphafold_wt"
sbatch --export NOTEBOOK_PATH,FEATURES_TO_USE --job-name=el3_all --time 24:00:00 --ntasks-per-node 44 --mem=180G --array=1-1 ../scripts/run_notebook_cpu.sh

export FEATURES_TO_USE="proteinsolver-protbert-rosetta_ddg-alphafold_wt-alphafold_change"
sbatch --export NOTEBOOK_PATH,FEATURES_TO_USE --job-name=el3_all_afchange --time 24:00:00 --ntasks-per-node 44 --mem=180G --array=1-1 ../scripts/run_notebook_cpu.sh


export FEATURES_TO_USE="proteinsolver"
sbatch --export NOTEBOOK_PATH,FEATURES_TO_USE --job-name=proteinsolver --time 24:00:00 --ntasks-per-node 44 --mem=180G --array=1-1 ../scripts/run_notebook_cpu.sh

export FEATURES_TO_USE="protbert"
sbatch --export NOTEBOOK_PATH,FEATURES_TO_USE --job-name=protbert --time 24:00:00 --ntasks-per-node 44 --mem=180G --array=1-1 ../scripts/run_notebook_cpu.sh

export FEATURES_TO_USE="rosetta_ddg"
sbatch --export NOTEBOOK_PATH,FEATURES_TO_USE --job-name=rosetta_ddg --time 24:00:00 --ntasks-per-node 44 --mem=180G --array=1-1 ../scripts/run_notebook_cpu.sh

export FEATURES_TO_USE="alphafold_wt"
sbatch --export NOTEBOOK_PATH,FEATURES_TO_USE --job-name=alphafold_wt --time 24:00:00 --ntasks-per-node 44 --mem=180G --array=1-1 ../scripts/run_notebook_cpu.sh

export FEATURES_TO_USE="alphafold_change"
sbatch --export NOTEBOOK_PATH,FEATURES_TO_USE --job-name=alphafold_change --time 24:00:00 --ntasks-per-node 44 --mem=180G --array=1-1 ../scripts/run_notebook_cpu.sh


```

---

## Imports

In [ ]:
import enum
import functools
import json
import os
import pickle
import shutil
import tempfile
import uuid
from collections import Counter
from pathlib import Path

import elaspic2 as el2
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import optuna
import optuna.integration.lightgbm as olgb
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from scipy import stats
from sklearn import metrics, model_selection
from sklearn.decomposition import PCA
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm

In [ ]:
pd.set_option("max_columns", 1000)
pd.set_option("max_rows", 1000)

## Parameters

In [ ]:
try:
    UNIQUE_ID
except NameError:
    UNIQUE_ID = str(uuid.uuid4())[:8]

UNIQUE_ID

In [ ]:
NOTEBOOK_DIR = Path("38_cagi6_sherloc_train_model").resolve()
NOTEBOOK_DIR.joinpath(UNIQUE_ID).mkdir(parents=True, exist_ok=True)

NOTEBOOK_DIR

In [ ]:
FEATURES_TO_USE = os.getenv("FEATURES_TO_USE")
TASK_ID = os.getenv("SLURM_ARRAY_TASK_ID")

FEATURES_TO_USE, TASK_ID

In [ ]:
DEBUG = TASK_ID is None

if DEBUG:
    FEATURES_TO_USE = "proteinsolver-protbert-rosetta_ddg-alphafold_wt"
else:
    assert FEATURES_TO_USE is not None

FEATURES_TO_USE, DEBUG

In [ ]:
class FeatureClass(enum.Enum):
    proteinsolver = "proteinsolver"
    protbert = "protbert"
    rosetta_ddg = "rosetta_ddg"
    alphafold_wt = "alphafold_wt"
    alphafold_change = "alphafold_change"

In [ ]:
features_to_use = [FeatureClass(fc) for fc in FEATURES_TO_USE.split("-")]

features_to_use

In [ ]:
feature_set_info_file = NOTEBOOK_DIR.joinpath(f"{FEATURES_TO_USE}.txt")
feature_set_info_file.touch()

feature_set_info_file

## Load results

In [ ]:
input_file_1 = NOTEBOOK_DIR.parent.joinpath(
    "37_cagi6_sherloc_combine_results", "combined-results.parquet"
)

input_file_2 = NOTEBOOK_DIR.parent.joinpath(
    "37_humsavar_combine_results", "combined-results.parquet"
)

input_file_1, input_file_2

In [ ]:
result_df = pd.concat(
    [pq.read_table(input_file_1).to_pandas(), pq.read_table(input_file_2).to_pandas()],
    ignore_index=True,
)

display(result_df.head(2))
print(len(result_df))

In [ ]:
effect_map = {
    "Uncertain significance": 0,
    "US": 0,
    "Likely benign": -1,
    "Benign": -1,
    "LB/B": -1,
    "Likely pathogenic": 1,
    "Pathogenic": 1,
    "LP/P": 1,
}

result_df["effect_score"] = result_df["effect"].map(effect_map)

In [ ]:
result_df["effect_score"].value_counts()

## Train ML models

### Prepare dataset

In [ ]:
scalar_features = (
    # MSA columns
    [c for c in result_df if c.startswith("msa")]
)

if FeatureClass.protbert in features_to_use:
    scalar_features += [
        "protbert_core_score_wt",
        "protbert_core_score_change",
    ]

if FeatureClass.proteinsolver in features_to_use:
    scalar_features += [
        "proteinsolver_core_score_wt",
        "proteinsolver_core_score_change",
    ]

if FeatureClass.rosetta_ddg in features_to_use:
    scalar_features += [c for c in result_df if "rosetta" in c]


if FeatureClass.alphafold_wt in features_to_use:
    scalar_features += [
        "alphafold_core_scores_residue_plddt_wt",
        "alphafold_core_scores_proten_ptm_wt",
    ]

if FeatureClass.alphafold_change in features_to_use:
    scalar_features += [
        "alphafold_core_scores_residue_plddt_change",
    ]


with NOTEBOOK_DIR.joinpath(UNIQUE_ID, "scalar-features.json").open("wt") as fout:
    json.dump(scalar_features, fout)

In [ ]:
vector_features = [
    "aa_wt_onehot",
    "aa_mut_onehot",
]

if FeatureClass.protbert in features_to_use:
    vector_features += [
        "protbert_core_features_residue_wt",
        "protbert_core_features_residue_change",
    ]

if FeatureClass.proteinsolver in features_to_use:
    vector_features += [
        "proteinsolver_core_features_residue_wt",
        "proteinsolver_core_features_residue_change",
    ]

if FeatureClass.alphafold_wt in features_to_use:
    vector_features += [
        "alphafold_core_features_residue_experimentally_resolved_wt",  # 0.19 [37]
        "alphafold_core_features_residue_predicted_lddt_wt",  # 0.17 [50]
        "alphafold_core_features_residue_msa_first_row_wt",  # 0.17 [256]
        "alphafold_core_features_residue_single_wt",  # 0.20 [384]
        "alphafold_core_features_residue_structure_module_wt",  # 0.18 [384]
    ]

if FeatureClass.alphafold_change in features_to_use:
    vector_features += [
        "alphafold_core_features_residue_experimentally_resolved_change",  # 0.11 [37]
        "alphafold_core_features_residue_predicted_lddt_change",  # 0.04 [50]
        "alphafold_core_features_residue_msa_first_row_change",  # 0.21 [256]
        "alphafold_core_features_residue_single_change",  # 0.15 [384]
        "alphafold_core_features_residue_structure_module_change",  # 0.05 [384]
    ]

with NOTEBOOK_DIR.joinpath(UNIQUE_ID, "vector-features.json").open("wt") as fout:
    json.dump(vector_features, fout)

In [ ]:
df = (
    result_df.dropna(
        subset=scalar_features
        + vector_features
        + [
            "effect_score",
        ]
    )
    .drop_duplicates(subset=["protein_id", "mutation"])
    .sort_values("protein_id")
    .reset_index(drop=True)
)
df = df[df["effect_score"] != 0]

# protein_map = {k: i for i, k in enumerate(df["protein_id"].unique())}
# groups = df["protein_id"].map(protein_map).values

value_counts = df["protein_id"].value_counts()
groups = df["protein_id"].drop_duplicates().map(value_counts).values

protein_ids = df["protein_id"]

X_ref = np.c_[
    df[scalar_features].values,
    np.hstack([np.vstack(df[col].values) for col in vector_features]),
]
X = X_ref
# X = X[:, important_features]

# low_confidence_mask = df["effect_score"] == 0

y = (df["effect_score"] > 0).values.astype(int)
# y[low_confidence_mask] = (df[low_confidence_mask]["el2_score"] > 2).values.astype(int)
# y = df["effect_score"].values

# weights = np.ones(len(df), dtype=np.float64)
# weights[df["effect_score"] == 1] = 0.5
# weights[df["effect_score"] == -1] = 0.5
# weights[low_confidence_mask] = 0.05

In [ ]:
len(df)

### Tune hyperparameters

In [ ]:
CONST_PARAM = {
    "objective": "binary",
#     "metric": "binary_logloss",
    "metric": "average_precision",
    # "is_unbalance": True,
}

In [ ]:
def training_loop(X, y, param, protein_ids, n_splits=6, progressbar=False):
    models = []
    preds = np.ones(len(y), dtype=np.float64) * np.nan
    gkf = GroupKFold(n_splits=n_splits)
    for train_index, test_index in tqdm(
        gkf.split(X, y, groups=protein_ids),
        total=n_splits,
        disable=not progressbar,
    ):
        X_training, X_testing = X[train_index], X[test_index]
        y_training, y_testing = y[train_index], y[test_index]
        # weights_training, weights_testing = weights[train_index], weights[test_index]

        dtrain = lgb.Dataset(
            X_training,
            label=y_training,
            # weight=weights_training,
        )
        model = lgb.train(param, dtrain)
        preds[test_index] = model.predict(X_testing)
        models.append(model)
    return models, preds

In [ ]:
def objective(trial, X, y):
    param = CONST_PARAM | {
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "num_leaves": trial.suggest_int("num_leaves", 2, 64),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 5, 60),
    }
    models, preds = training_loop(X, y, param, protein_ids)
    pred_labels = np.rint(preds)
    accuracy = metrics.accuracy_score(y, pred_labels)
    auc = metrics.roc_auc_score(y, preds)
    precision = metrics.average_precision_score(y, preds)
    return precision

In [ ]:
dtrain = lgb.Dataset(X, label=y, group=groups)

with tempfile.TemporaryDirectory() as model_dir:
    tuner = olgb.LightGBMTunerCV(
        CONST_PARAM | {"verbosity": -1},
        dtrain,
        verbose_eval=200,
        early_stopping_rounds=250,
        folds=GroupKFold(n_splits=6),
        num_boost_round=100,  # 1000
        model_dir=model_dir,
        time_budget=60 * 60 * 3,
        return_cvbooster=True,
    )
    tuner.run()
    booster = tuner.get_best_booster()

In [ ]:
param = tuner.best_params

print(tuner.best_score)
print(param)

In [ ]:
with NOTEBOOK_DIR.joinpath(UNIQUE_ID, "best-parameters-starting.json").open("wt") as fout:
    json.dump(param, fout)

In [ ]:
for bst_idx, bst in enumerate(booster.boosters):
    bst.save_model(str(NOTEBOOK_DIR.joinpath(UNIQUE_ID, f"model-starting-{bst_idx}.txt")))

# _ = dtrain.save_binary(str(NOTEBOOK_DIR.joinpath(UNIQUE_ID, "training-data.bin")))

### Feature elimination

In [ ]:
def get_feature_index(num_features, features_to_exclude):
    idx = 0
    seen = set(features_to_exclude)
    indices = list(reversed(range(num_features)))
    while indices:
        idx = indices.pop()
        while idx in seen:
            idx += 1
        seen.add(idx)
        yield idx


list(get_feature_index(10, {1, 2, 3, 9, 10}))

In [ ]:
feature_elimination_stats_file = NOTEBOOK_DIR.joinpath(
    UNIQUE_ID, "feature-elimination-stats.pickle"
)
feature_elimination_stats_file.touch()

features_to_exclude = {}

fe_round = -1
fe_round_stats = []
best_params = {cutoff: None for cutoff in [3000, 2000, 1000, 500] if X.shape[1] > cutoff}
highest_precision = None
while len(features_to_exclude) < X_ref.shape[1]:
    fe_round += 1

    # Apply feature elimination mask
    feature_mask = np.ones(X_ref.shape[1], dtype=bool)
    feature_mask[np.array(list(features_to_exclude), dtype=int)] = False
    X = X_ref[:, feature_mask]
    assert len(features_to_exclude) == X_ref.shape[1] - X.shape[1]

    # Retune parameters
    for cutoff, best_param in list(best_params.items()):
        if X.shape[1] < cutoff and best_param is None:
            dtrain = lgb.Dataset(X, label=y, group=groups)
            tuner = olgb.LightGBMTunerCV(
                CONST_PARAM | {"verbosity": -1},
                dtrain,
                verbose_eval=200,
                early_stopping_rounds=250,
                folds=GroupKFold(n_splits=6),
                num_boost_round=100,  # 1000
                time_budget=60 * 60 * 3,
            )
            tuner.run()
            best_params[cutoff] = tuner.best_params
            param = tuner.best_params
            with NOTEBOOK_DIR.joinpath(UNIQUE_ID, f"best-parameters-{cutoff}.json").open(
                "wt"
            ) as fout:
                json.dump(param, fout)

    # Re-train models
    models, preds = training_loop(X, y, param, protein_ids, progressbar=True)

    # Calculate model statistics
    corr = stats.spearmanr(y, preds)
    auc = metrics.roc_auc_score(y, preds)
    precision = metrics.average_precision_score(y, preds)

    # Save stats
    round_stats = (
        fe_round,
        corr[0],
        auc,
        precision,
        len(features_to_exclude),
        X.shape[1],
        list(features_to_exclude),
    )
    print(round_stats[:-1])
    fe_round_stats.append(round_stats)
    shutil.copyfile(
        feature_elimination_stats_file, feature_elimination_stats_file.with_suffix(".pickle.bak")
    )
    with feature_elimination_stats_file.open("wb") as fout:
        pickle.dump(fe_round_stats, fout, pickle.HIGHEST_PROTOCOL)

    # Save best models
    if highest_precision is None or precision > highest_precision:
        highest_precision = precision
        for model_idx, model in enumerate(models):
            model.save_model(
                str(NOTEBOOK_DIR.joinpath(UNIQUE_ID, f"model-{fe_round}-{model_idx}.txt"))
            )

    # Find new features to eliminate
    feature_importance_split = np.vstack(
        [model.feature_importance("split") for model in models]
    ).sum(axis=0)
    feature_importance_gain = np.vstack([model.feature_importance("gain") for model in models]).sum(
        axis=0
    )

    feature_df = pd.DataFrame(
        {
            "feature_idx": list(
                get_feature_index(len(feature_importance_split), features_to_exclude)
            ),
            "feature_importance_split": feature_importance_split,
            "feature_importance_gain": feature_importance_gain,
        }
    )

    if (feature_df["feature_importance_split"] == 0).any():
        feature_df = feature_df[feature_df["feature_importance_split"] == 0]
    else:
        num_features_to_drop = max(1, len(feature_df) // 100)
        feature_df = feature_df.sort_values(
            ["feature_importance_split", "feature_importance_gain"], ascending=True
        ).iloc[:num_features_to_drop]
    new_features_to_exclude = feature_df["feature_idx"].values.tolist()

    for feature_idx in new_features_to_exclude:
        assert feature_idx not in features_to_exclude
        features_to_exclude[feature_idx] = fe_round

In [ ]:
print("Done")